In [ ]:
api_key= 'api-key'

In [104]:
# importing libraries 
import speech_recognition as sr 
  
import os 
  
from pydub import AudioSegment 
from pydub.silence import split_on_silence 

In [105]:
import os
os.chdir('..')

In [106]:
print('Loading File')
  
# open the audio file stored in 
# the local system as a wav file. 
song = AudioSegment.from_wav('vero.WAV') 
song = song.normalize()
print('Open song')

# open a file where we will concatenate   
# and store the recognized text 
fh = open("recognized.txt", "w+") 
print('Writing text')

Loading File
Open song
Writing text


In [152]:
silence_level = song.dBFS-16

In [115]:
print('Creating Chunks')
# split track where silence is 0.5 seconds  
# or more and get chunks 
chunks = split_on_silence(song, 
    # must be silent for at least 0.5 seconds 
    # or 500 ms. adjust this value based on user 
    # requirement. if the speaker stays silent for  
    # longer, increase this value. else, decrease it. 
    min_silence_len = 1500, 

    # consider it silent if quieter than -16 dBFS 
    # adjust this per requirement 
    silence_thresh = silence_level
) 
print('Chunk size: '+str(len(chunks)))

# create a directory to store the audio chunks. 

try: 
    os.mkdir('audio_chunks') 
except(FileExistsError): 
    pass

# move into the directory to 
# store the audio files. 
os.chdir('audio_chunks') 

Creating Chunks
Chunk size: 62


In [141]:
i = 0
# process each chunk 
len_seconds = 0
output = ''
for chunk in chunks: 
    print('Trascribing chunk')
    len_seconds = len_seconds +chunk.duration_seconds 
    print(str(len_seconds))
    # Create 0.5 seconds silence chunk 
    chunk_silent = AudioSegment.silent(duration = 10)


    # add 0.5 sec silence to beginning and  
    # end of audio chunk. This is done so that 
    # it doesn't seem abruptly sliced. 
    audio_chunk = chunk_silent + chunk + chunk_silent 

    # export audio chunk and save it in  
    # the current directory. 
    print("saving chunk{0}.wav".format(i)) 
    # specify the bitrate to be 192 k 
    audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav") 

    # the name of the newly created chunk 
    filename = 'chunk'+str(i)+'.wav'

    print("Processing chunk "+str(i)) 

    # get the name of the newly created chunk 
    # in the AUDIO_FILE variable for later use. 
    file = filename 

    # create a speech recognition object 
    r = sr.Recognizer() 

    # recognize the chunk 
    with sr.AudioFile(file) as source: 
        # remove this if it is not working 
        # correctly. 
        r.adjust_for_ambient_noise(source) 
        audio_listened = r.listen(source) 

    # try converting it to text 
    rec = r.recognize_google(audio_listened, language = 'es', show_all=True)
    print(rec)
    # write the output to the file. 
    try:
        text = rec['alternative'][0]['transcript']
        output = output + text + ' '
    except:
        None


    i += 1

Trascribing chunk
0.969
saving chunk0.wav
Processing chunk 0
[]
Trascribing chunk
4.078
saving chunk1.wav
Processing chunk 1
{'alternative': [{'transcript': 'sobre', 'confidence': 0.83731335}, {'transcript': 'sobre mí'}, {'transcript': 'pobre'}, {'transcript': 'sobres'}, {'transcript': 'cobre'}], 'final': True}
Trascribing chunk
24.551
saving chunk2.wav
Processing chunk 2
{'alternative': [{'transcript': 'haciendo una', 'confidence': 0.5712527}, {'transcript': 'Kendo UNAM'}, {'transcript': 'viendo una'}, {'transcript': 'siendo una'}, {'transcript': 'vendo un ahh'}], 'final': True}
Trascribing chunk
25.483999999999998
saving chunk3.wav
Processing chunk 3
[]
Trascribing chunk
79.881
saving chunk4.wav
Processing chunk 4
{'alternative': [{'transcript': 'Estás estás estás', 'confidence': 0.848176}, {'transcript': 'estás estás'}, {'transcript': 'estas'}, {'transcript': 'estás'}, {'transcript': 'estás estas'}], 'final': True}
Trascribing chunk
85.334
saving chunk5.wav
Processing chunk 5
{'alte

In [148]:
os.chdir('../..')

In [149]:
!mkdir transcribed

In [153]:
text_file = open("transcribed/vero.txt", "w")
n = text_file.write(output)
text_file.close()

In [ ]:
#https://github.com/tyiannak/pyAudioAnalysis/wiki/5.-Segmentation